# Loading In Needed Modules

In [1]:
#Loading needed modules and classes/functions 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torchvision.datasets import ImageFolder 
from torchvision.io import read_image
from torchvision.io import decode_image
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import marvin
from marvin.tools.maps import Maps
from marvin.tools.image import Image
from marvin.utils.general.images import get_images_by_list
from marvin import config
from marvin.tools.cube import Cube
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix,r2_score
from sklearn.model_selection import train_test_split
import os
import shutil
from PIL import Image as image_PIL
import PIL 
from PIL import ImageShow
import time 
import hickle as hic
import seaborn as sns
sns.set(font_scale=1.4)
sns.set_style('ticks')

from marvin import config
from marvin.tools.image import Image
config.setRelease('DR15')
config.mode = 'local'
config.download = True


%matplotlib qt

[INFO]: No release version set. Setting default to DR15
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/redux/v2_4_3/drpall-v2_4_3.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/analysis/v2_4_3/2.2.1/dapall-v2_4_3-2.2.1.fits cannot be found. Setting dapall to None. (MarvinUserWarning)
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/redux/v2_4_3/drpall-v2_4_3.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /home/juanp/sas/mangawork/manga/spectro/analysis/v2_4_3/2.2.1/dapall-v2_4_3-2.2.1.fits cannot be found. Setting dapall to None. (MarvinUserWarning)


# Load in Data

In [2]:


data=pd.read_csv('CompleteTable.csv')  #Importing All MaNGA Data from DPRall Schema

galaxy_list=np.loadtxt('Query Results',dtype=str) #Pulling Manga ID's of galaxies which satisfy log(M) > 9 and 0 < z < 0.1


#Problem with image associated with manga id at galaxy_list[3548], mangaid- 1-135668
galaxy_list=np.delete(galaxy_list,3548)

galaxy_list=np.unique(galaxy_list)



galaxy_index=np.zeros(len(galaxy_list)) 
for i in range (len(galaxy_list)): #Getting the index of these galaxies in the schema table
    galaxy_index[i]=np.where(data.loc[:,'mangaid']==galaxy_list[i])[0][0]

galaxy_index=np.array(galaxy_index,dtype=int) #Ensuring we have array that can be used to index, force int 



galaxies=data.iloc[galaxy_index] #DF of galaxies which satisfies the condition, contains all relevant schema data 

galaxies=galaxies.sort_values(by=['plateifu']) #Sorting galaxies by plateifu to match ImageFolder Output 

#Creating the arrays of the independent variables were are interested in, and dependent variable n 

mass=galaxies.loc[:,'nsa_sersic_mass']
log_mass=np.log10(mass)

SFR=galaxies.loc[:,'sfr_tot']
log_SFR=np.log10(SFR)

ha_flux=galaxies.loc[:,'emline_gflux_tot_ha_6564']

mangaid=galaxies.loc[:,'mangaid']

plateifu=galaxies.loc[:,'plateifu']

if plateifu.all()==np.unique(plateifu).all() and mangaid.all()==np.unique(mangaid.all()):
    print('Is Unique List of Galaxies')
    
else:
    print('Try Again')


z=galaxies.loc[:,'z']

n=galaxies.loc[:,'nsa_sersic_n']
n=np.array(n,dtype=np.float32)
n=torch.from_numpy(n).to('cuda:0').reshape(-1,1)



Is Unique List of Galaxies


# Sanity Check Cell

In [6]:

#Sanity Check that Indexing is right 
# index_check=np.where(mangaid=='1-373175')[0][0]
index_check=3588
print(index_check)
print(galaxies.iloc[index_check])
print(mangaid.iloc[index_check])
print(log_SFR.iloc[index_check])

#1-177626

image=ImageFolder('/home/juanp/sas/dr15/manga/spectro/redux/v2_4_3/')
print(image[index_check])
PIL.ImageShow.show(image[index_check][0])

[WARNING]: subprocess 6500 is still running (ResourceWarning)


3588
plate                         9891
ifudesign                     9102
plateifu                 9891-9102
mangaid                   1-436646
drpallindx                   10915
                         ...      
sfr_1re                 0.00510849
sfr_tot                 0.00548693
htmID               15336624667054
nsa_elpetro_mass       4.65987e+10
nsa_sersic_mass         5.4664e+10
Name: 3455, Length: 622, dtype: object
1-436646
-2.26067034286715
(<PIL.Image.Image image mode=RGB size=562x562 at 0x7F820BBEE9D0>, 274)


1

In [5]:
print(len(np.unique(mangaid)))
print(len(image))
len(plateifu)

3589
3589


3589

In [5]:
# function to resize image
def resize_image(src_image, size=(128,128), bg_color="white"): 
    from PIL import Image, ImageOps 
    
    # resize the image so the longest dimension matches our target size
    src_image.thumbnail(size, Image.ANTIALIAS)
    
    # Create a new square background image
    new_image = Image.new("RGB", size, bg_color)
    
    # Paste the resized image into the center of the square background
    new_image.paste(src_image, (int((size[0] - src_image.size[0]) / 2), int((size[1] - src_image.size[1]) / 2)))
  
    # return the resized image
    return new_image


# Setting Up DataLoaders

In [6]:
img_size=(128,128)

# image=ImageFolder('/home/juanp/sas/dr15/manga/spectro/redux/v2_4_3/') #Picks up 3590 pictures, directory list has lenght 3637 however 

torch.manual_seed(15)
np.random.seed(15)

image_directory='/home/juanp/sas/dr15/manga/spectro/redux/v2_4_3'
classes= sorted(os.listdir(image_directory))



batch_size=50
image_copies=30

def load_dataset(data_path):
    # Load all the images
    transformation = transforms.Compose([
        # Randomly augment the image data
            # Random horizontal flip
        transforms.RandomHorizontalFlip(0.5),
            # Random vertical flip
        transforms.RandomVerticalFlip(0.3),
        #Rotates the image by some angle
        transforms.RandomRotation(0,360),
        # transform to tensors
        transforms.ToTensor(),
        # Normalize the pixel values (in R, G, and B channels)
        # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # Load all of the images, transforming them
    full_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        # transform=transformation
    )
    
    #This loop transforms the images and assigns them the correct label 

    full_dataset_v2 = []   
    for i in range(len(full_dataset)): 
        for j in range(image_copies): #This loops makes it so we have "image_copies" copies of each galaxy image with different orientations 
            temp=transformation(resize_image(full_dataset[i][0]))
            full_dataset_v2.append((temp,log_SFR.iloc[i],mangaid.iloc[i],z.iloc[i])) 
    
    full_dataset=full_dataset_v2

    print(len(full_dataset))


    # Split into training (70% and testing (30%) datasets)
    train_size = int(0.7 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    
    # use torch.utils.data.random_split for training/test split
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
    
    # define a loader for the training data we can iterate through in 50-image batches
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=False
    )
    
    # define a loader for the testing data we can iterate through in 50-image batches
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=False
    )
        
    return train_loader, test_loader




#####################################################################################################



# Get the iterative dataloaders for test and training data
train_loader, test_loader = load_dataset(image_directory)
batch_size = train_loader.batch_size
print("Data loaders ready to read", image_directory)



KeyboardInterrupt: 

# Defining Model Architecture 

In [5]:
# Create a neural net class
class Net(nn.Module):
    
    
    # Defining the Constructor
    def __init__(self, num_classes=3):
        super(Net, self).__init__()
        
        # In the init function, we define each layer we will use in our model
        
        # Our images are RGB, so we have input channels = 3. 
        # We will apply 12 filters in the first convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        
        # A second convolutional layer takes 12 input channels, and generates 24 outputs
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        
        # We in the end apply max pooling with a kernel size of 2
        self.pool = nn.MaxPool2d(kernel_size=2)
        
        # A drop layer deletes 20% of the features to help prevent overfitting
        self.drop = nn.Dropout2d(p=0.2)
        
        # Our 128x128 image tensors will be pooled twice with a kernel size of 2. 128/2/2 is 32.
        # This means that our feature tensors are now 128 x 128, and we've generated 24 of them
        
        # We need to flatten these in order to feed them to a fully-connected layer
        self.fc = nn.Linear(in_features=32 * 32 * 24, out_features=10)

        self.fc1= nn.Linear(in_features=10, out_features=1)


    def forward(self, x):
        # x=x+1
        
        # In the forward function, pass the data through the layers we defined in the init function
        # print(x)
        # Use a ReLU activation function after layer 1 (convolution 1 and pool)
        x=self.conv1(x)
        
        x=self.pool(x)
       

        x=F.relu(x)

        
      
 
     
        # Use a ReLU activation function after layer 2
        x = F.relu(self.pool(self.conv2(x))) 
        
        
        # Select some features to drop to prevent overfitting (only drop during training)
        x = F.dropout(self.drop(x), training=self.training)
        
        # Flatten
        x = x.view(-1, 32 * 32 * 24)
        # Feed to fully-connected layer to predict class
        x = self.fc(x)
        
        # Return class probabilities via a log_softmax function
        x=F.relu(x) 
        
        x= self.fc1(x)
        # print(x)
        return x
    
device = "cpu"
if (torch.cuda.is_available()):
    # if GPU available, use cuda (on a cpu, training will take a considerable length of time!)
    device = "cuda"

# Create an instance of the model class and allocate it to the device
model = Net(num_classes=len(SFR)).to('cuda')

print(model)

Net(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop): Dropout2d(p=0.2, inplace=False)
  (fc): Linear(in_features=24576, out_features=10, bias=True)
  (fc1): Linear(in_features=10, out_features=1, bias=True)
)


# Loading Up the Saved Model 

In [6]:
model.load_state_dict(torch.load('/home/juanp/Documents/SURP-2021/Models/SFR_Model_Rotation_30_Copies_Images'))
model.eval()

Net(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop): Dropout2d(p=0.2, inplace=False)
  (fc): Linear(in_features=24576, out_features=10, bias=True)
  (fc1): Linear(in_features=10, out_features=1, bias=True)
)

# Checking How Well the Model is Preforming 

In [13]:
all_truths_train = [] 
all_preds_train = [] 
all_mangaid_train=[]
all_z_train=[]
for (data,target,target1,target2) in train_loader:
    data, target = data.to(device), target.to(device)
    output=model(data)
    all_truths_train.append(target.cpu().detach().numpy())
    all_preds_train.append(output.cpu().detach().numpy()) 
    all_mangaid_train.append(target1)
    all_z_train.append(target2)


incomplete_batch_id_train=len(all_truths_train)-1

remainder_train=len(all_truths_train[incomplete_batch_id_train])

total_values_train=(len(all_truths_train)*batch_size)-(batch_size-remainder_train)



all_truths_train_array=np.zeros(total_values_train)
all_preds_train_array=np.zeros(total_values_train)
all_mangaid_train_array=[]
all_z_train_array=np.zeros(total_values_train)
k=0
while k < total_values_train:
    for i in range(len(all_truths_train)):
        if i<incomplete_batch_id_train:
            for j in range(batch_size):
                all_truths_train_array[k]=all_truths_train[i][j]
                all_preds_train_array[k]=all_preds_train[i][j]
                all_mangaid_train_array.append(all_mangaid_train[i][j])
                all_z_train_array[k]=all_z_train[i][j]
                k=k+1
                


        else:
            i=incomplete_batch_id_train
            for j in range(remainder_train):
                all_truths_train_array[k]=all_truths_train[i][j]
                all_preds_train_array[k]=all_preds_train[i][j]
                all_mangaid_train_array.append(all_mangaid_train[i][j])
                all_z_train_array[k]=all_z_train[i][j]
                k=k+1
                



# all_truths_train=all_truths_train_array
# all_preds_train=all_preds_train_array

#Sanity Check 
print(all_mangaid_train[6][44],all_truths_train[6][44])
print(all_mangaid_train_array[344],all_truths_train_array[344])


1-373175 -1.8767849197941102
1-373175 -1.8767849197941102


In [14]:
all_truths_test = [] 
all_preds_test = [] 
all_mangaid_test=[]
all_z_test=[]
for (data,target,target1,target2) in test_loader:
    data, target = data.to(device), target.to(device)
    output=model(data)
    all_truths_test.append(target.cpu().detach().numpy())
    all_preds_test.append(output.cpu().detach().numpy()) 
    all_mangaid_test.append(target1)
    all_z_test.append(target2)


incomplete_batch_id_test=len(all_truths_test)-1

remainder_test=len(all_truths_test[incomplete_batch_id_test])

total_values_test=(len(all_truths_test)*batch_size)-(batch_size-remainder_test)



all_truths_test_array=np.zeros(total_values_test)
all_preds_test_array=np.zeros(total_values_test)
all_mangaid_test_array=[]
all_z_test_array=np.zeros(total_values_test)
k=0
while k < total_values_test:
    for i in range(len(all_truths_test)):
        if i<incomplete_batch_id_test:
            for j in range(batch_size):
                all_truths_test_array[k]=all_truths_test[i][j]
                all_preds_test_array[k]=all_preds_test[i][j]
                all_mangaid_test_array.append(all_mangaid_test[i][j])
                all_z_test_array[k]=all_z_test[i][j]
                k=k+1
                


        else:
            i=incomplete_batch_id_test
            for j in range(remainder_test):
                all_truths_test_array[k]=all_truths_test[i][j]
                all_preds_test_array[k]=all_preds_test[i][j]
                all_mangaid_test_array.append(all_mangaid_test[i][j])
                all_z_test_array[k]=all_z_test[i][j]
                k=k+1
                



# all_truths_test=all_truths_test_array
# all_preds_test=all_preds_test_array


In [20]:
def MSE(pred,truth,n):
    return((1/n)*np.sum((pred-truth)**2))

np.savetxt('Train_Truths',all_truths_train_array)
np.savetxt('Train_Preds',all_preds_train_array)
np.savetxt('Train_Mangaid',all_mangaid_train_array,fmt='%s')
np.savetxt('Train_z',all_z_train_array)
np.savetxt('Test_Truths',all_truths_test_array)
np.savetxt('Test_Preds',all_preds_test_array)
np.savetxt('Test_Mangaid',all_mangaid_test_array,fmt='%s')
np.savetxt('Test_z',all_z_test_array)


In [21]:
r2_train=r2_score(all_truths_train_array,all_preds_train_array)
r2_test=r2_score(all_truths_test_array,all_preds_test_array)
MSE_train=MSE(all_preds_train_array,all_truths_train_array,len(all_preds_train_array))
MSE_test=MSE(all_preds_test_array,all_truths_test_array,len(all_truths_test_array))
# np.savetxt('Train_Data',all_truths_train_array)
# np.savetxt('Train_Label',all_preds_train_array)
# np.savetxt('Test_Data',all_truths_test_array)
# np.savetxt('Test_Label',all_preds_test_array)

plt.figure(figsize=(16,12))
plt.subplot(1,2,1) 
plt.title('Test Data Set',fontsize=14)
plt.scatter(all_truths_test_array, all_preds_test_array,color = 'b', alpha = 0.1*7/3)
plt.xlabel('Test True Values',fontsize=14)
plt.ylabel('Test Predicted Value',fontsize=14)
plt.text(-1.5,-5.5,'R$^2$='+ str(round(r2_test,4)),fontsize=14)
plt.text(-1.5,-6,'MSE='+str(round(MSE_test,4)),fontsize=14)
plt.plot([-5,1],[-5,1],'r--')

plt.subplot(1,2,2)
plt.title('Train Data Set',fontsize=14)
plt.scatter(all_truths_train_array, all_preds_train_array,color = 'k', alpha = 0.1)
plt.xlabel('Train True Values',fontsize=14)
plt.ylabel('Train Predicted Value',fontsize=14)
plt.plot([-5,1],[-5,1],'r--') 
plt.text(-1.5,-5.5,'R$^2$='+str(round(r2_train,4)),fontsize=14)
plt.text(-1.5,-6,'MSE='+str(round(MSE_train,4)),fontsize=14)
plt.show()
# plt.savefig('/home/juanp/Documents/SURP-2021/Plots/Model 3/MSE Rotations 30 Copies of Images.pdf')

# Making the Summary Plots

In [8]:
all_images_train = [] 
for (data,target,target1,target2) in train_loader:
    data = data.to(device)
    all_images_train.append(data.detach().cpu().numpy())
    


incomplete_batch_id_train=len(all_images_train)-1

remainder_train=len(all_images_train[incomplete_batch_id_train])

total_values_train=(len(all_images_train)*batch_size)-(batch_size-remainder_train)




all_images_train_array=[]
# k=0
# while k < total_values_train:
for i in range(len(all_images_train)):
    if i<incomplete_batch_id_train:
        for j in range(batch_size):
            all_images_train_array.append(np.dstack(all_images_train[i][j][0:3]))
            # k=k+1
            


    else:
        i=incomplete_batch_id_train
        for j in range(remainder_train):
            all_images_train_array.append(np.dstack(all_images_train[i][j][0:3]))
            # k=k+1
                



# all_images_train=all_images_train_array

# np.savetxt('Images_Train', all_images_train_array)
hic.dump(all_images_train_array,'Images_Train')

In [7]:
all_images_test = [] 
for (data,target,target1,target2) in test_loader:
    data = data.to(device)
    all_images_test.append(data.detach().cpu().numpy())
    


incomplete_batch_id_test=len(all_images_test)-1

remainder_test=len(all_images_test[incomplete_batch_id_test])

total_values_test=(len(all_images_test)*batch_size)-(batch_size-remainder_test)




all_images_test_array=[]
# k=0
# while k < total_values_test:
for i in range(len(all_images_test)):
    if i<incomplete_batch_id_test:
        for j in range(batch_size):
            all_images_test_array.append(np.dstack(all_images_test[i][j][0:3]))
            # k=k+1
            


    else:
        i=incomplete_batch_id_test
        for j in range(remainder_test):
            all_images_test_array.append(np.dstack(all_images_test[i][j][0:3]))
            # k=k+1
                



# all_images_test=all_images_test_array

# np.savetxt('Images_Test', all_images_test_array)
hic.dump(all_images_test_array,'Images_Test')

In [2]:
all_truths_train_array=np.loadtxt('Train_Truths')
all_preds_train_array=np.loadtxt('Train_Preds')
all_mangaid_train_array=np.loadtxt('Train_Mangaid',dtype=str)
all_z_train_array=np.loadtxt('Train_z')
all_truths_test_array=np.loadtxt('Test_Truths')
all_preds_test_array=np.loadtxt('Test_Preds')
all_mangaid_test_array=np.loadtxt('Test_Mangaid',dtype=str)
all_z_test_array=np.loadtxt('Test_z')

all_images_train_array=hic.load('Images_Train')
all_images_test_array=hic.load('Images_Test')

ValueError: Provided argument 'file_obj' does not appear to be a valid hickle file! (Unable to open file (file signature not found))

In [3]:
def delta(truths,preds):
    return(np.abs(truths-preds))

delta_train=delta(all_truths_train_array,all_preds_train_array)
delta_test=delta(all_truths_test_array,all_preds_test_array)


In [9]:
# print(np.min(delta_train),np.max(delta_train),np.mean(delta_train))
# print(np.min(delta_test),np.max(delta_test),np.mean(delta_test))


# print(np.shape(np.where(all_mangaid_test_array=='1-392010')))

# 1-37922 -3.161576348521903

# print(all_mangaid_train_array[986],all_truths_train_array[986])

a=np.where(mangaid=='1-37922')
print(a)
print(SFR.iloc[a])

# plt.imshow(all_images_train_array[986])



(array([229]),)
2879    0.000689
Name: sfr_tot, dtype: float64


In [38]:
# print(all_images_train[0][0].shape)
# print(all_images_train_array[0].shape)


x_loc=70
y_loc_truth=105
y_loc_pred=110
y_loc_delta=115
y_loc_z=120
y_loc_mangaid=125

#Index to chose a galaxy at random 
# i_1=np.random.choice(len(all_truths_train_array))
# i_2=np.random.choice(len(all_truths_train_array))
# i_3=np.random.choice(len(all_truths_train_array))
# i_4=np.random.choice(len(all_truths_train_array)) 
# i_5=np.random.choice(len(all_truths_train_array))  
# i_6=np.random.choice(len(all_truths_train_array)) 
# i_7=np.random.choice(len(all_truths_train_array)) 
# i_8=np.random.choice(len(all_truths_test_array)) 
# i_9=np.random.choice(len(all_truths_test_array)) 
# i_10=np.random.choice(len(all_truths_test_array)) 

#Finding the galaxies where the model preformed the best 
# best_index_train=np.argsort(delta_train)[:7]
# best_index_test=np.argsort(delta_test)[:3]

# i_1=best_index_train[0]
# i_2=best_index_train[1]
# i_3=best_index_train[2]
# i_4=best_index_train[3] 
# i_5=best_index_train[4]  
# i_6=best_index_train[5] 
# i_7=best_index_train[6] 
# i_8=best_index_test[0] 
# i_9=best_index_test[1] 
# i_10=best_index_test[2] 

#Finding the galaxies where the model preformed the worst 

worst_index_train=np.argsort(-delta_train)[:7]
worst_index_test=np.argsort(-delta_test)[:3]

i_1=worst_index_train[0]
i_2=worst_index_train[1]
i_3=worst_index_train[2]
i_4=worst_index_train[3] 
i_5=worst_index_train[4]  
i_6=worst_index_train[5] 
i_7=worst_index_train[6] 
i_8=worst_index_test[0] 
i_9=worst_index_test[1] 
i_10=worst_index_test[2] 



plt.figure()
plt.rc('font',size=7)
plt.subplot(2,5,1)
plt.imshow(all_images_train_array[i_1])
plt.text(x_loc,y_loc_truth,'True log SFR='+str(round(all_truths_train_array[i_1],5)),color='white')
plt.text(x_loc,y_loc_pred,'Pred log SFR='+str(round(all_preds_train_array[i_1],5)),color='white')
plt.text(x_loc,y_loc_mangaid,'Manga-id:'+all_mangaid_train_array[i_1],color='white')
plt.text(x_loc,y_loc_z,'z:'+str(round(all_z_train_array[i_1],5)),color='white')
plt.text(x_loc,y_loc_delta,'Delta:'+str(round(delta_train[i_1],5)),color='white')
plt.subplot(2,5,2)
plt.imshow(all_images_train_array[i_2])
plt.text(x_loc,y_loc_truth,'True log SFR='+str(round(all_truths_train_array[i_2],5)),color='white')
plt.text(x_loc,y_loc_pred,'Pred log SFR='+str(round(all_preds_train_array[i_2],5)),color='white')
plt.text(x_loc,y_loc_mangaid,'Manga-id:'+all_mangaid_train_array[i_2],color='white')
plt.text(x_loc,y_loc_z,'z:'+str(round(all_z_train_array[i_2],5)),color='white')
plt.text(x_loc,y_loc_delta,'Delta:'+str(round(delta_train[i_2],5)),color='white')
plt.subplot(2,5,3)
plt.imshow(all_images_train_array[i_3])
plt.text(x_loc,y_loc_truth,'True log SFR='+str(round(all_truths_train_array[i_3],5)),color='white')
plt.text(x_loc,y_loc_pred,'Pred log SFR='+str(round(all_preds_train_array[i_3],5)),color='white')
plt.text(x_loc,y_loc_mangaid,'Manga-id:'+all_mangaid_train_array[i_3],color='white')
plt.text(x_loc,y_loc_z,'z:'+str(round(all_z_train_array[i_3],5)),color='white')
plt.text(x_loc,y_loc_delta,'Delta:'+str(round(delta_train[i_3],5)),color='white')
plt.subplot(2,5,4)
plt.imshow(all_images_train_array[i_4])
plt.text(x_loc,y_loc_truth,'True log SFR='+str(round(all_truths_train_array[i_4],5)),color='white')
plt.text(x_loc,y_loc_pred,'Pred log SFR='+str(round(all_preds_train_array[i_4],5)),color='white')
plt.text(x_loc,y_loc_mangaid,'Manga-id:'+all_mangaid_train_array[i_4],color='white')
plt.text(x_loc,y_loc_z,'z:'+str(round(all_z_train_array[i_4],5)),color='white')
plt.text(x_loc,y_loc_delta,'Delta:'+str(round(delta_train[i_4],5)),color='white')
plt.subplot(2,5,5)
plt.imshow(all_images_train_array[i_5])
plt.text(x_loc,y_loc_truth,'True log SFR='+str(round(all_truths_train_array[i_5],5)),color='white')
plt.text(x_loc,y_loc_pred,'Pred log SFR='+str(round(all_preds_train_array[i_5],5)),color='white')
plt.text(x_loc,y_loc_mangaid,'Manga-id:'+all_mangaid_train_array[i_5],color='white')
plt.text(x_loc,y_loc_z,'z:'+str(round(all_z_train_array[i_5],5)),color='white')
plt.text(x_loc,y_loc_delta,'Delta:'+str(round(delta_train[i_5],5)),color='white')
plt.subplot(2,5,6)
plt.imshow(all_images_train_array[i_6])
plt.text(x_loc,y_loc_truth,'True log SFR='+str(round(all_truths_train_array[i_6],5)),color='white')
plt.text(x_loc,y_loc_pred,'Pred log SFR='+str(round(all_preds_train_array[i_6],5)),color='white')
plt.text(x_loc,y_loc_mangaid,'Manga-id:'+all_mangaid_train_array[i_6],color='white')
plt.text(x_loc,y_loc_z,'z:'+str(round(all_z_train_array[i_6],5)),color='white')
plt.text(x_loc,y_loc_delta,'Delta:'+str(round(delta_train[i_6],5)),color='white')
plt.subplot(2,5,7)
plt.imshow(all_images_train_array[i_7])
plt.text(x_loc,y_loc_truth,'True log SFR='+str(round(all_truths_train_array[i_7],5)),color='white')
plt.text(x_loc,y_loc_pred,'Pred log SFR='+str(round(all_preds_train_array[i_7],5)),color='white')
plt.text(x_loc,y_loc_mangaid,'Manga-id:'+all_mangaid_train_array[i_7],color='white')
plt.text(x_loc,y_loc_z,'z:'+str(round(all_z_train_array[i_7],5)),color='white')
plt.text(x_loc,y_loc_delta,'Delta:'+str(round(delta_train[i_7],5)),color='white')
plt.subplot(2,5,8)
plt.imshow(all_images_test_array[i_8])
plt.text(x_loc,y_loc_truth,'True log SFR='+str(round(all_truths_test_array[i_8],5)),color='white')
plt.text(x_loc,y_loc_pred,'Pred log SFR='+str(round(all_preds_test_array[i_8],5)),color='white')
plt.text(x_loc,y_loc_mangaid,'Manga-id:'+all_mangaid_test_array[i_8],color='white')
plt.text(x_loc,y_loc_z,'z:'+str(round(all_z_test_array[i_8],5)),color='white')
plt.text(x_loc,y_loc_delta,'Delta:'+str(round(delta_test[i_8],5)),color='white')
plt.show()
plt.subplot(2,5,9)
plt.imshow(all_images_test_array[i_9])
plt.text(x_loc,y_loc_truth,'True log SFR='+str(round(all_truths_test_array[i_9],5)),color='white')
plt.text(x_loc,y_loc_pred,'Pred log SFR='+str(round(all_preds_test_array[i_9],5)),color='white')
plt.text(x_loc,y_loc_mangaid,'Manga-id:'+all_mangaid_test_array[i_9],color='white')
plt.text(x_loc,y_loc_z,'z:'+str(round(all_z_test_array[i_9],5)),color='white')
plt.text(x_loc,y_loc_delta,'Delta:'+str(round(delta_test[i_9],5)),color='white')
plt.show()
plt.subplot(2,5,10)
plt.imshow(all_images_test_array[i_10])
plt.text(x_loc,y_loc_truth,'True log SFR='+str(round(all_truths_test_array[i_10],5)),color='white')
plt.text(x_loc,y_loc_pred,'Pred log SFR='+str(round(all_preds_test_array[i_10],5)),color='white')
plt.text(x_loc,y_loc_mangaid,'Manga-id:'+all_mangaid_test_array[i_10],color='white')
plt.text(x_loc,y_loc_z,'z:'+str(round(all_z_test_array[i_10],5)),color='white')
plt.text(x_loc,y_loc_delta,'Delta:'+str(round(delta_test[i_10],5)),color='white')
plt.show()